# Instalación y datos

## Instalación de postgres y carga de datos

In [1]:
# install
!apt update
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# pin package versions for google colab compatibility
!pip install SQLAlchemy==1.4.46
!pip install ipython-sql==0.4.1
# set connection
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres
%sql create schema control2;
%sql set search_path to control2;

from google.colab import output
output.clear()

### Airports

In [2]:
%%sql

CREATE TABLE airports(
    id bigint,
    name varchar(128),
    city varchar(128),
    country varchar(64),
    iata_code varchar(3),
    icao_code varchar(4),
    latitude decimal(9, 6),
    longitude decimal(9, 6),
    altitude float,
    timezone decimal(4, 1),
    dst varchar(1),
    tz varchar(64),
    airport_type varchar(8),
    source varchar(16)
);

 * postgresql+psycopg2://@/postgres


""


In [3]:
!wget https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat

--2023-04-28 18:46:09--  https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1127225 (1.1M) [text/plain]
Saving to: ‘airports.dat’

airports.dat        100%[===================>]   1.07M  --.-KB/s    in 0.06s   

2023-04-28 18:46:10 (18.6 MB/s) - ‘airports.dat’ saved [1127225/1127225]



In [4]:
%%sql

COPY airports FROM '/content/airports.dat' WITH (FORMAT CSV, DELIMITER(','), NULL '\N' )

 * postgresql+psycopg2://@/postgres


""


### Airlines

In [5]:
%%sql

CREATE TABLE airlines(
    id bigint,
    name varchar(128),
    alias varchar(64),
    iata_code varchar(4),
    icao_code varchar(8),
    callsign varchar(128),
    country varchar(64),
    active varchar(1)
);

 * postgresql+psycopg2://@/postgres


""


In [6]:
!wget https://raw.githubusercontent.com/jpatokal/openflights/master/data/airlines.dat

--2023-04-28 18:46:10--  https://raw.githubusercontent.com/jpatokal/openflights/master/data/airlines.dat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 396896 (388K) [text/plain]
Saving to: ‘airlines.dat’

airlines.dat        100%[===================>] 387.59K  --.-KB/s    in 0.04s   

2023-04-28 18:46:10 (9.30 MB/s) - ‘airlines.dat’ saved [396896/396896]



In [7]:
%%sql

COPY airlines FROM '/content/airlines.dat' WITH (FORMAT CSV, DELIMITER(','), NULL '\N' )

 * postgresql+psycopg2://@/postgres


""


### Routes

In [8]:
%%sql

CREATE TABLE routes(
    airline_code varchar(3),
    airline_id bigint,
    source_airport_code varchar(4),
    source_airport_id bigint,
    destination_airport_code varchar(4),
    destination_airport_id bigint,
    vcodeshare varchar(1),
    codeshare boolean,
    stops int,
    equipment varchar(128)
);

 * postgresql+psycopg2://@/postgres


""


In [9]:
!wget https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat

--2023-04-28 18:46:11--  https://raw.githubusercontent.com/jpatokal/openflights/master/data/routes.dat
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2377148 (2.3M) [text/plain]
Saving to: ‘routes.dat’

routes.dat          100%[===================>]   2.27M  --.-KB/s    in 0.08s   

2023-04-28 18:46:11 (28.9 MB/s) - ‘routes.dat’ saved [2377148/2377148]



In [10]:
%%sql

COPY routes(
    airline_code,
    airline_id,
    source_airport_code,
    source_airport_id,
    destination_airport_code,
    destination_airport_id,
    vcodeshare,
    stops,
    equipment
) FROM '/content/routes.dat' WITH (FORMAT CSV, DELIMITER(','), NULL '\N' );

UPDATE routes
SET codeshare = FALSE
WHERE vcodeshare = '';

UPDATE routes
SET codeshare = TRUE
WHERE vcodeshare = 'Y';

ALTER TABLE routes DROP COLUMN vcodeshare;

 * postgresql+psycopg2://@/postgres


""


# Esquema

## Esquema

La base de datos tiene 3 tablas: `airlines`, `airports` y `routes`. Puedes revisar el esquema en detalle [en esta página](https://openflights.org/data.html) de dónde sacamos los datos.

* `airlines(id, name, alias, iata_code, icao_code, callsign, country, active)`
* `airports(id, name, city, country, iata_code, icao_code, latitude, longitude, altitude, timezone, dst, tz, airport_type, source)`
* `routes(airline_code, airline_id, source_airport_code, source_airport_id, destination_airport_code, destination_airport_id, codeshare, stops, equipment)`

# Preguntas

Haga consultas en SQL para obtener la siguiente información:

### 1. Indique los pares de aeropuertos con más vuelos de uno al otro


In [ ]:
%%

### 2. Enumere todos las ciudades que conectan Santiago con Frankfurt


### 3. ¿Cuál es son los nombres de las aerolineas más grande? (La que tiene la mayor cantidad de rutas). Liste las 10 más grandes.

### 4. ¿Cuáles son los 5 aeropuertos más importantes? (Aeropuertos de los cuáles llegan y salen más rutas)

### 5. Entregue todas las rutas (sin repetición) que van desde New York hacia Berlin y pasan por exactamente tres aeropuertos. Para esto entregue tres columnas con los códigos de cada uno de los aeropuertos involucrados.

### 6. Entregue los 10 aeropuertos más cercanos al aeropuerto 'Pdte. Carlos Ibañez del Campo Airport' (sin incluirlo), ordenados en distancia de forma ascendente.

Para calcular la distancia, puede utilizar la siguiente fórmula:

$$
distancia(p1, p2) = \sqrt{(p2.lat - p1.lat)^{2} + (p2.long - p1.long)^{2}}
$$

Es importante mencionar que esta fórmula no es precisa, pues no considera la curvatura de la tierra 😉